In [1]:
class treeNode:
    def __init__(self,nameValue,numOccur,parentNode):
        self.name=nameValue
        self.count=numOccur
        self.nodeLink=None
        self.parent=parentNode
        self.children={}
    def inc(self,numOccur):
        self.count+=numOccur
    def disp(self,ind=1):
        print ' '*ind,self.name, ' ',self.count
        for child in self.children.values():
            child.disp(ind+1)

In [2]:
def updateHeader(nodeToTest,targetNode):
    while nodeToTest.nodeLink!=None:
        nodeToTest=nodeToTest.nodeLink
    nodeToTest.nodeLink=targetNode

In [3]:
def updateTree(items,inTree,headerTable,count):
    if items[0] in inTree.children:
        inTree.children[items[0]].inc(count)
    else:
        inTree.children[items[0]]=treeNode(items[0],count,inTree)
        if headerTable[items[0]][1]==None:
            headerTable[items[0]][1]=inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1],inTree.children[items[0]])
    if len(items)>1:
        updateTree(items[1::],inTree.children[items[0]],headerTable,count)

In [40]:
def createTree(dataSet,minSup=1):
    headerTable={}
    for trans in dataSet:
        for item in trans:
            headerTable[item]=headerTable.get(item,0)+dataSet[trans]
    for k in headerTable.keys():
        if headerTable[k]<minSup:
            del(headerTable[k])
    freqItemSet=set(headerTable.keys())
    if len(freqItemSet)==0:
        return None,None
    for k in headerTable:
        headerTable[k]=[headerTable[k],None]
    retTree=treeNode('Null Set',1,None)
    for tranSet,count in dataSet.items():
        localD={}
        for item in tranSet:
            if item in freqItemSet:
                localD[item]=headerTable[item][0]
        if len(localD)>0:
            orderedItems=[v[0] for v in sorted(localD.items(),key=lambda p:p[1],reverse=True)]
            updateTree(orderedItems,retTree,headerTable,count)
    return retTree,headerTable

In [5]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

In [6]:
def createInitSet(dataSet):
    retDict={}
    for trans in dataSet:
        retDict[frozenset(trans)]=1
    return retDict

In [7]:
simpDat=loadSimpDat()

In [8]:
initSet=createInitSet(simpDat)

In [9]:
myFPtree,myHeaderTab=createTree(initSet,3)

In [10]:
myFPtree.disp()

  Null Set   1
   x   1
    s   1
     r   1
   z   5
    x   3
     y   3
      s   2
       t   2
      r   1
       t   1
    r   1


In [11]:
def ascendTree(leafNode,prefixPath):
    if leafNode.parent!=None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent,prefixPath)

In [12]:
def findPrefixPath(basePat,treeNode):
    condPats={}
    while treeNode!=None:
        prefixPath=[]
        ascendTree(treeNode,prefixPath)
        if len(prefixPath)>1:
            condPats[frozenset(prefixPath[1:])]=treeNode.count
        treeNode=treeNode.nodeLink
    return condPats

In [13]:
myHeaderTab

{'r': [3, <__main__.treeNode instance at 0x103dd6320>],
 's': [3, <__main__.treeNode instance at 0x103dd6200>],
 't': [3, <__main__.treeNode instance at 0x103dd6248>],
 'x': [4, <__main__.treeNode instance at 0x103dd6170>],
 'y': [3, <__main__.treeNode instance at 0x103dd61b8>],
 'z': [5, <__main__.treeNode instance at 0x103dd6128>]}

In [16]:
findPrefixPath('x',myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [19]:
findPrefixPath('z',myHeaderTab['z'][1])

{}

In [23]:
findPrefixPath('r',myHeaderTab['r'][1])

{frozenset({'s', 'x'}): 1, frozenset({'z'}): 1, frozenset({'x', 'y', 'z'}): 1}

In [29]:
def mineTree(inTree,headerTable,minSup,preFix,freqItemList):
    bigL=[v[0] for v in sorted(headerTable.items(),key=lambda p:p[1])]
    for basePat in bigL:
        newFreqSet=preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        condPattBases=findPrefixPath(basePat,headerTable[basePat][1])
        myCondTree,myHead=createTree(condPattBases,minSup)
        if myHead!=None:
            print 'conditional tree for: ',newFreqSet
            myCondTree.disp(1)
            mineTree(myCondTree,myHead,minSup,newFreqSet,freqItemList)

In [30]:
freqItems=[]

In [31]:
mineTree(myFPtree,myHeaderTab,3,set([]),freqItems)

conditional tree for:  set(['y'])
  Null Set   1
   x   3
    z   3
conditional tree for:  set(['y', 'z'])
  Null Set   1
   x   3
conditional tree for:  set(['s'])
  Null Set   1
   x   3
conditional tree for:  set(['t'])
  Null Set   1
   y   3
    x   3
     z   3
conditional tree for:  set(['x', 't'])
  Null Set   1
   y   3
conditional tree for:  set(['z', 't'])
  Null Set   1
   y   3
    x   3
conditional tree for:  set(['x', 'z', 't'])
  Null Set   1
   y   3
conditional tree for:  set(['x'])
  Null Set   1
   z   3


In [32]:
freqItems

[{'y'},
 {'x', 'y'},
 {'y', 'z'},
 {'x', 'y', 'z'},
 {'s'},
 {'s', 'x'},
 {'t'},
 {'t', 'y'},
 {'t', 'x'},
 {'t', 'x', 'y'},
 {'t', 'z'},
 {'t', 'y', 'z'},
 {'t', 'x', 'z'},
 {'t', 'x', 'y', 'z'},
 {'r'},
 {'x'},
 {'x', 'z'},
 {'z'}]

In [33]:
parsedDat=[line.split() for line in open('kosarak.dat').readlines()]

In [34]:
initSet=createInitSet(parsedDat)

In [35]:
myFPtree,myHeaderTab=createTree(initSet,100000)

In [36]:
myFreqList=[]

In [41]:
mineTree(myFPtree,myHeaderTab,100000,set([]),myFreqList)

conditional tree for:  set(['1'])
  Null Set   1
   6   123785
conditional tree for:  set(['3'])
  Null Set   1
   11   11129
   6   214487
    11   135153
conditional tree for:  set(['11', '3'])
  Null Set   1
   6   135153
conditional tree for:  set(['11'])
  Null Set   1
   6   301164


In [42]:
myFreqList

[{'1'},
 {'1'},
 {'1', '6'},
 {'3'},
 {'11', '3'},
 {'11', '3', '6'},
 {'3', '6'},
 {'11'},
 {'11', '6'},
 {'6'}]